In [1]:
import time
import pandas as pd
import numpy as np
import colorama as col
from tqdm import tqdm
import joblib
from joblib import Parallel,delayed
import datetime as dt
import fxcmpy
import pyttsx3
import datetime as dt
from Live import *
from librairies.strategy import *
from librairies.bt import *
from sklearn.metrics import accuracy_score, make_scorer, precision_score, recall_score, precision_recall_curve, confusion_matrix, classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler,MaxAbsScaler,quantile_transform,PolynomialFeatures
engine = pyttsx3.init()
print('version fxcmpy :',fxcmpy.__version__)
#print('version socketio :',socketio.__version__)

Importing Librairies...
version fxcmpy : 1.2.6


In [2]:
_token = 'dbdc379ce7761772c662c3e92250a0ae38385b2c'
_server = 'demo'
_user_id = 'D261282181'
_compte = '01215060'
_password = 'waXz1'

_period = 'm15'
_name = 'MLPClassifier'

TICKER_LIST = ['GBP/AUD']
x = TICKER_LIST[0]
TIK = ['JPY','CHF','CAD','GBP','AUD','NZD','SEK','NOK','MXN','ZAR','TRY','ILS','CNH','USD','HKD']
RATE = [105.4,0.8989,1.276,0.877,0.7676,0.7201,8.3684,8.5238,20.09,14.81,7.03,3.28,6.458,1,7.7527]
df_ratefx = pd.DataFrame(index=TIK)
df_ratefx['rate'] = RATE
_scaler = MaxAbsScaler()
#con = fxcmpy.fxcmpy(access_token=_token, log_level='error',server=_server)

In [3]:
def conX():
    con = fxcmpy.fxcmpy(access_token=_token, log_level='error',server=_server)
    if con.is_connected() == True:
        print(col.Fore.GREEN+'Connexion établie'+col.Style.RESET_ALL)
        print('Compte utilisé : ',con.get_account_ids())
    else:
        print(col.Fore.RED+'Connexion non établie'+col.Style.RESET_ALL)
    return(con)

def deconX(con):
    con = con.close()
    if con.is_connected() == True:
        print(col.Fore.GREEN+'Connexion non intérrompue'+col.Style.RESET_ALL)
        print('Compte utilisé : ',con.get_account_ids())
    else:
        print(col.Fore.RED+'Connexion intérrompue'+col.Style.RESET_ALL)
    return()

def scrap_hist(ticker,invers = 'non'):
    #_debut = pd.to_datetime((dt.datetime.now()-dt.timedelta(minutes=3987165)).strftime('%Y-%m-%d'))
    #_fin = pd.to_datetime((dt.datetime.now().strftime('%Y-%m-%d')))
    data = con.get_candles(ticker,period=_period,start=_debut,end=_fin)
    data['Open'] = (data['bidopen']+data['askopen'])/2
    data['High'] = (data['bidhigh']+data['askhigh'])/2
    data['Low'] = (data['bidlow']+data['asklow'])/2
    data['Close'] = (data['bidclose']+data['askclose'])/2
    return(data)

def buy(df_all):
    print(dt.datetime.now())
    ##### BUY 
    _price = round(con.get_candles(x,period='m15',number=1).askclose[-1],5)
    _time = con.get_candles(x,period='m15',number=1).index[-1]
    _amount = 200
    _limit = round(_price + _price * 0.004,5)
    _stop = round(_price  - _price * 0.002,5)
    _atmarket = 3
    order = con.open_trade(symbol=x,is_buy=True, is_in_pips=False, amount=20, time_in_force='IOC',order_type='MarketRange',limit=_limit,stop=_stop, at_market=3)
    print(" Bougie de l'opération d'éxecution",col.Fore.BLUE,_time,col.Style.RESET_ALL)
    print(col.Fore.GREEN,'Achat sur le ticker',col.Fore.YELLOW,x,col.Fore.GREEN,'demandé à ',col.Fore.CYAN,_price,col.Style.RESET_ALL)

def sell(df_all):
    print(dt.datetime.now())
    _atmarket = 3
    _price = round(con.get_candles(x,period='m15',number=1).bidclose[-1],5)
    _amount = 200
    _stop = round(_price + _price * 0.002,5)
    _limit = round(_price  - _price * 0.004,5)
    order = con.open_trade(symbol=x,is_buy=False, is_in_pips=False, amount=20, time_in_force='IOC',order_type='MarketRange',limit=_limit,stop=_stop, at_market=3)
    print(" Bougie de l'opération d'éxecution",col.Fore.BLUE,_time,col.Style.RESET_ALL)
    print(col.Fore.RED,'Vente sur le ticker',col.Fore.YELLOW,x,col.Fore.RED,'demandé à ',col.Fore.CYAN,_price,col.Style.RESET_ALL)

    #expiration = (dt.datetime.now() + dt.timedelta(hours=4)).strftime(format='%Y-%m-%d %H:%M'))
    return()

def close():
    con.close_all_for_symbol(x)
    return()

def init_df():
    _path = 'JOBLIB/Ticker_'+_period+'/df_'+x.replace('/','')
    df_all = joblib.load('JOBLIB/Built_bases/df_all')
    df_all = df_all[df_all.Symbol==x.replace('/','')]
    df_all = df_all[['Close','CloseAsk','CloseBid','High','HighAsk','HighBid','Low','LowAsk','LowBid','Open','OpenAsk','OpenBid','Symbol','Date']]
    _fin = dt.datetime.now()
    _deb = df_all.index[-1]
    _debut = dt.datetime(_deb.year,_deb.month,_deb.day,_deb.hour,_deb.minute)
    addon = con.get_candles(x,period='m15',start=_debut,end=_fin).drop(['tickqty'],axis=1)
    addon = addon.rename(columns={'bidopen':'OpenBid','bidclose':'CloseBid','bidhigh':'HighBid','bidlow':'LowBid','askopen':'OpenAsk','askclose':'CloseAsk','askhigh':'HighAsk','asklow':'LowAsk'})
    addon['Open'] = (addon.OpenAsk + addon.OpenBid)/2
    addon['High'] = (addon.HighAsk + addon.HighBid)/2
    addon['Low'] = (addon.LowAsk + addon.LowBid)/2
    addon['Close'] = (addon.CloseAsk + addon.CloseBid)/2
    addon['Symbol'] = x.replace('/','')
    addon['Date'] = addon.index
    addon['Date'] = pd.to_datetime(addon['Date'].dt.strftime(date_format='%Y-%m-%d'))
    df_all = df_all.append(addon.iloc[1:-1,:])
    df_all['WE'] = np.where(((df_all.index.weekday == 5) | (df_all.index.weekday == 6)),None,df_all.index.weekday)
    df_all = df_all.dropna()
    df_all =df_all.drop(['WE'],axis=1)
    joblib.dump(df_all,_path)
    return(df_all)

In [4]:
___Author___='LumberJack Jyss'
print('Global Optimized LumberJack Environment Motor for For_Ex\nLumberJack Jyss 5781(c)')
print(col.Fore.BLUE,'°0Oo_D.A.G._26_oO0°')
print(col.Fore.YELLOW,col.Back.BLUE,'--- Bigfoot 1. #v0.60 ---',col.Style.RESET_ALL)


print('')
engine.say(" Initialization of Bigfoot 1, FX system")
engine.say("Bigfoot's Connexion to the a p i")
engine.runAndWait()

try:
    con.is_connected() == True
    
    engine.say("already Connected")
    engine.runAndWait()
    print(col.Fore.GREEN+'Connexion rétablie'+col.Style.RESET_ALL)
    print('Compte utilisé : ',con.get_account_ids())
    print('')
    
except:
    try:
        con = conX()
        con.is_connected() == True
        print(col.Fore.GREEN+'Connexion établie'+col.Style.RESET_ALL)
        print('Compte utilisé : ',con.get_account_ids())
        engine.say("Bigfoot is Connected")
        engine.runAndWait()
    except:
        print(col.Fore.RED+'Connexion non établie'+col.Style.RESET_ALL)
        engine.say("Mayday, mayday, Not Connected, mauzerfucker!")
        engine.say("Check your internet, and launch agin the Bigfoot")
        engine.runAndWait()
        print('')
        #os._exit(0)
        con = deconX()
        time.sleep(1)
        con = conX()
print('\rChargement de la base...',end='',flush=True)
engine.say("Ignition of Bigfoot. Loading the database.")
engine.runAndWait()
#_path = 'JOBLIB/Ticker_'+_period+'/df_'+x.replace('/','')
#df_all = joblib.load(_path)
#df_all = df_all[df_all.Symbol==x.replace('/','')]
#df_all = df_all[['Close','CloseAsk','CloseBid','High','HighAsk','HighBid','Low','LowAsk','LowBid','Open','OpenAsk','OpenBid','Symbol','Date']]
#engine.say("Database is loaded. Ready to enter Live")
#engine.runAndWait()
print('\rBase Chargée.',end='',flush=True)
while True:
    engine.say("Building the base")
    engine.runAndWait()
    print('\nConstruction de la base...')
    ##########
    df_all = init_df()

    
    if len(df_all) > 19000000000000:
        df_all = df_all.iloc[-190000:,:]

    if _period == 'H1':
        df_all = timerange1D(df_all)
        daily_all = get_daily(df_all,TICKER_LIST)

    ##### Si la période n'est pas H1, on récupère d'abord les data en 1H pour tous les tickers, et on construit la base daily à partir du 1H => daily_all
    else:
        _period='H1'
        
        df_all = timerange1D(df_all)
        hourly_all = get_all_data(TICKER_LIST,_period='H1')
        _fin = dt.datetime.now()
        _deb = hourly_all.index[-1]
        _debut = dt.datetime(_deb.year,_deb.month,_deb.day,_deb.hour)
        hourly_add = con.get_candles(x,period='H1',start=_debut,end=_fin).drop(['tickqty'],axis=1) # df_all[df_all.index.minute==0] # scrap_hist(x)
        hourly_add = hourly_add.rename(columns={'bidopen':'OpenBid','bidclose':'CloseBid','bidhigh':'HighBid','bidlow':'LowBid','askopen':'OpenAsk','askclose':'CloseAsk','askhigh':'HighAsk','asklow':'LowAsk'})
        hourly_add['Open'] = (hourly_add.OpenAsk + hourly_add.OpenBid)/2
        hourly_add['High'] = (hourly_add.HighAsk + hourly_add.HighBid)/2
        hourly_add['Low'] = (hourly_add.LowAsk + hourly_add.LowBid)/2
        hourly_add['Close'] = (hourly_add.CloseAsk + hourly_add.CloseBid)/2
        hourly_add['Symbol'] = x.replace('/','')
        hourly_all = hourly_all.append(hourly_add.iloc[1:-1,:])
        hourly_all = timerange1D(hourly_all)
        _period='m15'
        daily_all = get_daily(hourly_all,TICKER_LIST)
        #del hourly_all
    daily_all = timerange1W(daily_all)
    weekly_all = get_weekly(daily_all,TICKER_LIST)
    daily_all = adr(daily_all,_window=14)
    df_all = getadr(daily_all,df_all,TICKER_LIST)
    df_all = adrhnl(daily_all,df_all,TICKER_LIST)
    df_all = sma(df_all=df_all,_window=200)
    df_all = bollinger(df_all,_slow=20)
    df_all = slowstochastic(df_all,TICKER_LIST)
    df_all = ema(df_all,21,TICKER_LIST)
    df_all = ema(df_all,8,TICKER_LIST)
    weekly_all = pivot(weekly_all,TICKER_LIST)
    df_all = pivotimportdf(df_all,weekly_all,TICKER_LIST)
    df_all = atr(df_all,TICKER_LIST,14)
    df_all = rvi(df_all,TICKER_LIST,_window=14)
    df_all = sbgamma(df_all,TICKER_LIST)
    df_all = onhisma(df_all,TICKER_LIST,_window=5)
    df_all = onlosma(df_all,TICKER_LIST,_window=5)
    df_all = onhisma(df_all,TICKER_LIST,_window=21)
    df_all = onlosma(df_all,TICKER_LIST,_window=21)
    df_all = onhisma(df_all,TICKER_LIST,_window=34)
    df_all = onlosma(df_all,TICKER_LIST,_window=34)
    df_all = importohlc(df_all,weekly_all,TICKER_LIST,_suffix='_weekly')
    df_all = importohlc(df_all=df_all,other_all=daily_all,TICKER_LIST=TICKER_LIST,_suffix='_daily')
    df_all = stochastic(df_all)
    
    ##### Signal is from strategy. This is potential good one. But we have to create the TRACKER column where the Signal where efficient

    # Proceed an MaxAbsScaler on features
    
    ##### On backtest selon le ticker selectionné sur la période déterminée
    _fin = df_all.index[-1]
    _debut = df_all.index[0]
    print(x)
    
    _nb_bougie_exit = 2000000
    _trigger_reengage = 0
    _trigger_target = 1
    _trigger_invers = 0
    _trigger_sl = 1
    _verbose = 0
    _cash_ini = 200000
    _rate = 1/df_ratefx.loc[x[4:],'rate']
    backtest = df_all[df_all.Symbol == x.replace('/','')]
    _target =  0.004
    _exposure = 10
    _size = 400000
    _sl =  0.002
    _open_hour = 8 # day only
    _close_hour = 23 # day only
    _window = 0 # day only
    
    print()
    print(col.Fore.CYAN,'###############################################################################################')
    print(' #################################### TRAIN/TEST WITHOUT AI ####################################')
    print(' ###############################################################################################',col.Style.RESET_ALL)
    _year_bottom = _debut
    _year_top = _fin

    TRACKER,_nb_looser = bt(backtest,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
            _rate,x,_target,_exposure,_size,_sl)

    print()
    print(col.Fore.BLUE,'###############################################################################################')
    print(' #################################### DENOISING & ENHANCING ####################################')
    print(' ###############################################################################################',col.Style.RESET_ALL)
    while _nb_looser > 0 :
        
        df_all['TRACKER'] = np.where(df_all.index.isin(TRACKER),1,0)
        df_all['Valid'] = np.where(((df_all.Signal!=0)&(df_all.TRACKER==1)),1,0)
        df_all['Signal'] = np.where(((df_all.Valid==1)&(df_all.Signal==1)),1,np.where(((df_all.Valid==1)&(df_all.Signal==-1)),-1,0))
        backtest = df_all[df_all.Symbol == x.replace('/','')]
        ##### Purification of signal by denoising and enhancing
        TRACKER,_nb_looser = bt(backtest,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
                _rate,x,_target,_exposure,_size,_sl)

    
    print('Featuring...')
   
    features = featuring(df_all)

    features = features.dropna()


    features['TRACKER'] = np.where(features.index.isin(TRACKER),1,0)

    features['Valid'] = np.where(((features.Signal!=0)&(features.TRACKER==1)),1,0) # Don't miss the point that even a Signal -1 must be considered as a good one by TRACKER

    _break = int(len(features)*0.7)
    features_train = features.iloc[:_break,:]
    features_test = features.iloc[_break:,:]

    features_train = scaling(features_train,scaler=_scaler)

    features_test = scaling(features_test,scaler=_scaler)

    features_train = quantile(features_train,quantile_transform)

    features_test = quantile(features_test,quantile_transform)


    print(col.Fore.BLUE,'                          DEEP LEARNING',col.Style.RESET_ALL)
    _name = 'MLPClassifier'
    _model = MLPClassifier(hidden_layer_sizes=(300,10), activation='relu', solver='adam', alpha=0.000001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=800, shuffle=True, random_state=26, tol=0.0000001, verbose=False, warm_start=True, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.8, beta_2=0.999, epsilon=1e-08)

    _model.fit(features_train.drop(['Date','Symbol','TRACKER','Valid','Signal'],axis=1),features_train.Valid)
    yhat = _model.predict(features_test.drop(['Date','Symbol','TRACKER','Valid','Signal'],axis=1))
    accu = round(accuracy_score(features_test.Valid, yhat) * 100,2)
    prec = round(precision_score(features_test.Valid, yhat,pos_label=1) * 100,2)
    recall = round(recall_score(features_test.Valid, yhat) * 100,2)
    f1 = round(f1_score(features_test.Valid, yhat) * 100,2)


    print('Signaux - Accuracy :' ,accu,'%')
    print('Signaux - Precision :',prec,'%')
    print('Signaux - Recall :', recall,'%')
    print('Achat - F-measure: :' ,f1,'%')
    print('\n')
    print(classification_report(features_test.Valid, yhat, target_names=['Neg', 'Pos']))
    conf_matrix = pd.DataFrame(columns=['Bonnes_Estimations','Mauvaises_Estimations'])
    tt = confusion_matrix(features_test.Valid, yhat, labels=[1,0])    #_model.classes_)

    conf_matrix.loc['vrais-réels'] = tt[0]
    conf_matrix.loc['faux-réels'] = tt[1]
    
    print(conf_matrix)
    print()
    print(col.Fore.BLUE,'Signaux pour',col.Fore.YELLOW,x,col.Style.RESET_ALL)
    _tp = tt[0][0]
    _fn = tt[0][1]
    _prec = round((tt[0][0]/(tt[0][0]+tt[0][1]))*100,2)
    _rec = round((tt[0][1]/(tt[0][0]+tt[0][1]))*100,2)

    print('Vrais signaux trouvés    : ',tt[0][0])
    print('Vrais signaux non trouvé :',tt[0][1])
    print('Total des vrais signaux  :',tt[0][0]+tt[0][1])
    if _prec > 44 and prec > 44 :
        print(col.Fore.GREEN)
    elif _prec < 18 or prec < 18 :
        print(col.Fore.RED)
    else:
        print(col.Fore.YELLOW)
    print('Precision : ',_prec,'%')
    if _rec > 60 and _rec > 60 :
        print(col.Fore.GREEN)
    elif _rec < 40 or _rec < 40 :
        print(col.Fore.RED)
    else:
        print(col.Fore.YELLOW)
    print('Recall',_rec,'%')
    print(col.Style.RESET_ALL)

    
    ##### On backtest selon le ticker selectionné sur la période déterminée

    print(x)
    
    print('Training done.\nWaiting for the candle...')
    print()

    ##########

    while dt.datetime.now().minute not in [0,15,30,45]:
        print('\rTicker tracké :',x,' ',dt.datetime.now(),end='',flush=True)
        time.sleep(1)
    print()
    while con.get_candles(x,period=_period,start=dt.datetime(df_all.index[-1].year,df_all.index[-1].month,df_all.index[-1].day,df_all.index[-1].hour,df_all.index[-1].minute)\
          ,end=dt.datetime.now()).index[-1].minute != dt.datetime.now().minute:

          time.sleep(0.5)
        
    _fin = dt.datetime.now()
    _deb = df_all.index[-1]
    _debut = dt.datetime(_deb.year,_deb.month,_deb.day,_deb.hour,_deb.minute)
    addon = con.get_candles(x,period='m15',start=_debut,end=_fin).drop(['tickqty'],axis=1)
    addon = addon.rename(columns={'bidopen':'OpenBid','bidclose':'CloseBid','bidhigh':'HighBid','bidlow':'LowBid','askopen':'OpenAsk','askclose':'CloseAsk','askhigh':'HighAsk','asklow':'LowAsk'})
    addon['Open'] = (addon.OpenAsk + addon.OpenBid)/2
    addon['High'] = (addon.HighAsk + addon.HighBid)/2
    addon['Low'] = (addon.LowAsk + addon.LowBid)/2
    addon['Close'] = (addon.CloseAsk + addon.CloseBid)/2
    addon['Symbol'] = x.replace('/','')
    addon['Date'] = addon.index
    addon['Date'] = pd.to_datetime(addon['Date'].dt.strftime(date_format='%Y-%m-%d'))
    df_all = df_all[['Close','CloseAsk','CloseBid','High','HighAsk','HighBid','Low','LowAsk','LowBid','Open','OpenAsk','OpenBid','Symbol','Date']]
    df_all = df_all.append(addon.iloc[1:-1,:])
    #df_all = df_all.iloc[-263570:,:]
    

    ##### Si la période demandée est déjà H1, on peut construire directement la base daily pour tous les tickers => daily_all
    if _period == 'H1':
        df_all = timerange1D(df_all)
        daily_all = get_daily(df_all,TICKER_LIST)

    ##### Si la période n'est pas H1, on récupère d'abord les data en 1H pour tous les tickers, et on construit la base daily à partir du 1H => daily_all
    else:
        _period='H1'
        df_all = timerange1D(df_all)
        hourly_all = get_all_data(TICKER_LIST,_period='H1')
        _fin = dt.datetime.now()
        _deb = hourly_all.index[-1]
        _debut = dt.datetime(_deb.year,_deb.month,_deb.day,_deb.hour)
        hourly_add = con.get_candles(x,period='H1',start=_debut,end=_fin).drop(['tickqty'],axis=1) # df_all[df_all.index.minute==0] # scrap_hist(x)
        hourly_add = hourly_add.rename(columns={'bidopen':'OpenBid','bidclose':'CloseBid','bidhigh':'HighBid','bidlow':'LowBid','askopen':'OpenAsk','askclose':'CloseAsk','askhigh':'HighAsk','asklow':'LowAsk'})
        hourly_add['Open'] = (hourly_add.OpenAsk + hourly_add.OpenBid)/2
        hourly_add['High'] = (hourly_add.HighAsk + hourly_add.HighBid)/2
        hourly_add['Low'] = (hourly_add.LowAsk + hourly_add.LowBid)/2
        hourly_add['Close'] = (hourly_add.CloseAsk + hourly_add.CloseBid)/2
        hourly_add['Symbol'] = x.replace('/','')
        hourly_all = hourly_all.append(hourly_add.iloc[1:-1,:])
        hourly_all = timerange1D(hourly_all)
        _period='m15'
        daily_all = get_daily(hourly_all,TICKER_LIST)
        #del hourly_all
    daily_all = timerange1W(daily_all)
    weekly_all = get_weekly(daily_all,TICKER_LIST)
    daily_all = adr(daily_all,_window=14)
    df_all = getadr(daily_all,df_all,TICKER_LIST)
    df_all = adrhnl(daily_all,df_all,TICKER_LIST)
    df_all = sma(df_all=df_all,_window=200)
    df_all = bollinger(df_all,_slow=20)
    df_all = slowstochastic(df_all,TICKER_LIST)
    df_all = ema(df_all,21,TICKER_LIST)
    df_all = ema(df_all,8,TICKER_LIST)
    weekly_all = pivot(weekly_all,TICKER_LIST)
    df_all = pivotimportdf(df_all,weekly_all,TICKER_LIST)
    df_all = atr(df_all,TICKER_LIST,14)
    df_all = rvi(df_all,TICKER_LIST,_window=14)
    df_all = sbgamma(df_all,TICKER_LIST)
    df_all = onhisma(df_all,TICKER_LIST,_window=5)
    df_all = onlosma(df_all,TICKER_LIST,_window=5)
    df_all = onhisma(df_all,TICKER_LIST,_window=21)
    df_all = onlosma(df_all,TICKER_LIST,_window=21)
    df_all = onhisma(df_all,TICKER_LIST,_window=34)
    df_all = onlosma(df_all,TICKER_LIST,_window=34)
    df_all = importohlc(df_all,weekly_all,TICKER_LIST,_suffix='_weekly')
    df_all = importohlc(df_all=df_all,other_all=daily_all,TICKER_LIST=TICKER_LIST,_suffix='_daily')
    
    df_all = stochastic(df_all)
    features = featuring(df_all)

    # And drop the nan
    features = features.dropna()
    ##### Signal is from strategy. This is potential good one. But we have to create the TRACKER column where the Signal where efficient

    # Proceed an MaxAbsScaler on features
    features = scaling(features,scaler=_scaler)

    features = quantile(features,quantile_transform)

    _valid = _model.predict(features.drop(['Date','Symbol','Signal'],axis=1))[-1]

    _signal = df_all.Signal[-1]

    print('\nTest sur la bougie',features.index[-1])
    if _valid == 1 and _signal == 1 :
        buy()

    elif _valid == 1 and _signal == -1 :
        sell()

    else:
        print(col.Fore.BLUE,'\nNO SIGNAL FOR',col.Fore.YELLOW,x,'\n',col.Style.RESET_ALL)
    
    print()
   
    df_all = df_all[['Close','CloseAsk','CloseBid','High','HighAsk','HighBid','Low','LowAsk','LowBid','Open','OpenAsk','OpenBid','Symbol','Date']]  
    #df_all = joblib.load(_path)


Global Optimized LumberJack Environment Motor for For_Ex
LumberJack Jyss 5781(c)
 °0Oo_D.A.G._26_oO0°
  --- Bigfoot 1. #v0.60 --- 

Connexion établie
Compte utilisé :  [1277536]
Connexion établie
Compte utilisé :  [1277536]
Base Chargée.
Construction de la base...
GBP/AUD

 ###############################################################################################
 #################################### TRAIN/TEST WITHOUT AI ####################################
 ############################################################################################### 
Librairies imported

Début des opérations horodatée à 2021-02-17 09:50:21.886629

Chargement de la nouvelle base


 Le rate du ticker GBP/AUD est à  1.302761855132882 
Bases chargées
TETEL process effectué
 ENTERING THE BACKTEST 
100%|██████████| 267602/267602 [00:15<00:00, 16967.03it/s]
 For ticker  GBP/AUD 
 
Total Profit & Loss : $  -1756878.58 . En  5460   transactions.
 
Winners Number : 1700 
 
Loosers number : 3760 
BT's ex

In [6]:
%%time

df_all = init_df()

CPU times: user 4.05 s, sys: 4.07 s, total: 8.12 s
Wall time: 8.69 s
